In [2]:
# Importations
import pandas as pd
from pandasql import sqldf
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn import linear_model
import statsmodels.api as sm


In [3]:
# getting data from external file:
monthly_data = pd.read_sas("./CA.sas7bdat")
return_data = pd.read_sas("./returndata.sas7bdat")
ff_data = pd.read_sas("./factors_monthly.sas7bdat")

In [4]:
#print (monthly_data.head())
print(ff_data.head())

        date   mktrf     smb     hml      rf    year  month  umd     dateff
0 1926-07-01  0.0296 -0.0256 -0.0243  0.0022  1926.0    7.0  NaN 1926-07-31
1 1926-08-01  0.0264 -0.0117  0.0382  0.0025  1926.0    8.0  NaN 1926-08-31
2 1926-09-01  0.0036 -0.0140  0.0013  0.0023  1926.0    9.0  NaN 1926-09-30
3 1926-10-01 -0.0324 -0.0009  0.0070  0.0032  1926.0   10.0  NaN 1926-10-30
4 1926-11-01  0.0253 -0.0010 -0.0051  0.0031  1926.0   11.0  NaN 1926-11-30


In [5]:
#merge based on PERMNO
monthly_data["monthid"] = (monthly_data.date.dt.year-2000)*12 + monthly_data.date.dt.month
ff_data["monthid"] = (ff_data.date.dt.year-2000)*12 + ff_data.date.dt.month
return_data["monthid"] = (return_data.DATE.dt.year-2000)*12 + return_data.DATE.dt.month
ff_data = ff_data[(ff_data['monthid']>=1) & (ff_data['monthid']<=299) ]
monthly_data = monthly_data.rename(columns={'permno': 'PERMNO'})
print(return_data.columns)
mergedf = pd.merge(monthly_data,return_data,how="left", on=["PERMNO","monthid"])
merged = pd.merge(mergedf, ff_data,how="left",on=["monthid"])
print(merged.columns)

Index(['PERMNO', 'DATE', 'RET', 'monthid'], dtype='object')
Index(['PERMNO', 'date_x', 'yyyymm', 'ret', 'ret_t1', 'lnsize', 'bk2mkt', 'ep',
       'beta', 'ivol', 'CUSIP', 'COMNAM', 'TICKER', 'PRC', 'SHROUT', 'monthid',
       'me', 'gvkey', 'DATADATE', 'ATQ', 'CEQQ', 'EPSPXQ', 'IBQ', 'SALEQ',
       'DATE', 'RET', 'date_y', 'mktrf', 'smb', 'hml', 'rf', 'year', 'month',
       'umd', 'dateff'],
      dtype='object')


In [6]:
# Declaration: Online resources were used to ensure clearity of this.
# Credit: ChatGPT: asked for a cleaner version of groupby("PERNMO")

def compound_return(returns):
    # returns: a numpy array of returns
    return (1 + returns).prod() - 1

merged['MOM'] = (
    merged.groupby("PERMNO")["ret"]
      .apply(lambda x: x.rolling(window=12, min_periods=10)
                   .apply(compound_return, raw=True))
      .reset_index(level=0, drop=True)
)


In [59]:
def winsorize_series(s, lower=0.01, upper=0.99):
    return s.clip(lower=s.quantile(lower), upper=s.quantile(upper))

In [60]:


merged['MOM_winsorize'] = winsorize_series(merged['MOM'])
merged['bk2mkt_winsorize'] = winsorize_series(merged['bk2mkt'])
merged['lnsize_winsorize'] = winsorize_series(merged['lnsize'])
merged['ep_winsorize'] = winsorize_series(merged['ep'])
merged['beta_winsorize'] = winsorize_series(merged['beta'])
merged['ivol_winsorize'] = winsorize_series(merged['ivol'])

print(merged['ep'].std())
print(merged['lnsize_winsorize'].std())
print(merged['bk2mkt_winsorize'].std())
print(merged['ep_winsorize'].std())
print(merged['beta_winsorize'].std())
print(merged['ivol_winsorize'].std())
print("")
print(merged['MOM'].std())
print(merged['lnsize'].std())
print(merged['bk2mkt'].std())
print(merged['ep'].std())
print(merged['beta'].std())
print(merged['ivol'].std())


0.043885755117170834
1.6281686076534982
0.2906433282955342
0.018272399983715378
0.7365577842493983
0.010575730630191871

0.7876318612427587
1.6751919220228833
0.37727926332245254
0.043885755117170834
0.7999771455954879
0.011511088926617672


In [8]:
def summary_statistic(array):
    print("number of observation",len(array))
    print("mean:",np.mean(array))
    print("std: ",np.std(array))
    print("min:",np.min(array))
    print("max:",np.max(array))
    print("1st percentile", np.percentile(array,1))
    print("99st percentile", np.percentile(array,99))
    print("Maximum", np.max(array))



In [9]:
summary_statistic(merged['MOM'].dropna())

number of observation 21863
mean: 0.2442010116841693
std:  0.7876138481414522
min: -0.9722955122681691
max: 53.662785856291336
1st percentile -0.6399789813149054
99st percentile 2.2192849928771516
Maximum 53.662785856291336


In [105]:
def compute_factor_quintile(merged, factor):
    # Define column names based on the factor.
    winsor_col = f"{factor}_winsorize"      # e.g., "ep_winsorize"
    
    # Create the winsorized quintile variable by grouping by DATE.
    merged[f"{factor}_rank"] = merged.groupby("monthid")[winsor_col].transform(
        lambda x: pd.qcut(x, q=5, labels=False, duplicates="drop") + 1
    )
    merged.sort_values(["monthid", f"{factor}_rank"], inplace=True)
    
    # Group by the monthid.
    merged[f"monthid_{factor}"] = merged["monthid"].astype(str) + merged[f"{factor}_rank"].astype(str)
    groups = merged.groupby([f"monthid_{factor}"])
    
    # Initialize dictionary to accumulate quintile summary statistics.
    quintile = {
        "monthid": [],
        f"{factor}_rank": [],
        "ret_t1": [],
        factor:[]
    }
    
    # Loop over each group (each unique month/quintile combination).
    for name, group in groups:
        quintile["monthid"].append(group["monthid"].iloc[0])
        # Using the computed rank directly:
        quintile[f"{factor}_rank"].append(group[f"{factor}_rank"].iloc[0])
        quintile["ret_t1"].append(group['ret_t1'].mean())
        quintile[factor].append(group[factor].mean())
    
    # Create a DataFrame from the aggregated statistics and sort.
    quintile_df = pd.DataFrame(quintile)
    quintile_df.sort_values(["monthid", f"{factor}_rank"], inplace=True)
    quintile_df.reset_index(drop=True, inplace=True)

    return quintile_df


In [11]:
factors = ['ep','lnsize','bk2mkt','MOM','beta','ivol']
quintile_port = []

print(merged.columns)
for f in factors:
    df = compute_factor_quintile(merged,f)
    quintile_port.append(df)
    # add to excel
    print(df.head())

Index(['PERMNO', 'date_x', 'yyyymm', 'ret', 'ret_t1', 'lnsize', 'bk2mkt', 'ep',
       'beta', 'ivol', 'CUSIP', 'COMNAM', 'TICKER', 'PRC', 'SHROUT', 'monthid',
       'me', 'gvkey', 'DATADATE', 'ATQ', 'CEQQ', 'EPSPXQ', 'IBQ', 'SALEQ',
       'DATE', 'RET', 'date_y', 'mktrf', 'smb', 'hml', 'rf', 'year', 'month',
       'umd', 'dateff', 'MOM', 'MOM_winsorize', 'bk2mkt_winsorize',
       'lnsize_winsorize', 'ep_winsorize', 'beta_winsorize', 'ivol_winsorize'],
      dtype='object')
   monthid  ep_rank    ret_t1        ep
0        1      1.0  0.503356 -0.011105
1        1      2.0  0.218565  0.003041
2        1      3.0  0.271324  0.005944
3        1      4.0  0.149769  0.011681
4        1      5.0 -0.010883  0.049185
   monthid  lnsize_rank    ret_t1     lnsize
0        1          1.0  0.519571   5.846272
1        1          2.0  0.196213   7.706301
2        1          3.0  0.087781   8.660386
3        1          4.0  0.150513   9.441580
4        1          5.0  0.114786  11.356380
   mont

In [36]:
merged.columns

Index(['PERMNO', 'date_x', 'yyyymm', 'ret', 'ret_t1', 'lnsize', 'bk2mkt', 'ep',
       'beta', 'ivol', 'CUSIP', 'COMNAM', 'TICKER', 'PRC', 'SHROUT', 'monthid',
       'me', 'gvkey', 'DATADATE', 'ATQ', 'CEQQ', 'EPSPXQ', 'IBQ', 'SALEQ',
       'DATE', 'RET', 'date_y', 'mktrf', 'smb', 'hml', 'rf', 'year', 'month',
       'umd', 'dateff', 'MOM', 'MOM_winsorize', 'bk2mkt_winsorize',
       'lnsize_winsorize', 'ep_winsorize', 'beta_winsorize', 'ivol_winsorize',
       'ep_rank', 'monthid_ep', 'lnsize_rank', 'monthid_lnsize', 'bk2mkt_rank',
       'monthid_bk2mkt', 'MOM_rank', 'monthid_MOM', 'beta_rank',
       'monthid_beta', 'ivol_rank', 'monthid_ivol'],
      dtype='object')

In [14]:
def hedge_creation(quintlile,factor, long,short):
    long_leg = quintlile.loc[quintlile[f"{factor}_rank"] == long]
    short_leg = quintlile.loc[quintlile[f"{factor}_rank"] == short]
    
    hedge_port1 = pd.merge(long_leg,short_leg,on=['monthid'])
    hedge_port1['hedge_ret'] = hedge_port1['ret_t1_x'] - hedge_port1['ret_t1_y']
    #print(f"long leg for {factor}:",len(long_leg))
    #print(len(short_leg))
    hedge_port1 = pd.merge(hedge_port1,ff_data,on=['monthid'])
    return hedge_port1

In [15]:
merged.columns

Index(['PERMNO', 'date_x', 'yyyymm', 'ret', 'ret_t1', 'lnsize', 'bk2mkt', 'ep',
       'beta', 'ivol', 'CUSIP', 'COMNAM', 'TICKER', 'PRC', 'SHROUT', 'monthid',
       'me', 'gvkey', 'DATADATE', 'ATQ', 'CEQQ', 'EPSPXQ', 'IBQ', 'SALEQ',
       'DATE', 'RET', 'date_y', 'mktrf', 'smb', 'hml', 'rf', 'year', 'month',
       'umd', 'dateff', 'MOM', 'MOM_winsorize', 'bk2mkt_winsorize',
       'lnsize_winsorize', 'ep_winsorize', 'beta_winsorize', 'ivol_winsorize',
       'ep_rank', 'monthid_ep', 'lnsize_rank', 'monthid_lnsize', 'bk2mkt_rank',
       'monthid_bk2mkt', 'MOM_rank', 'monthid_MOM', 'beta_rank',
       'monthid_beta', 'ivol_rank', 'monthid_ivol'],
      dtype='object')

In [ ]:
quintile_port[5][quintile_port[5].ivol_rank == 5]

,monthid,ivol_rank,ret_t1,ivol
4,1,5.0,0.530817,0.057428
9,2,5.0,-0.010420,0.069087
14,3,5.0,-0.087577,0.078528
19,4,5.0,-0.106266,0.062836
25,5,5.0,0.204035,0.053835
...,...,...,...,...
1526,295,5.0,0.015626,0.030773
1531,296,5.0,0.086352,0.033971
1536,297,5.0,0.007661,0.027782
1541,298,5.0,0.126239,0.026609


In [97]:
factors = ['ep','lnsize','bk2mkt','MOM','beta','ivol']
hedge_port = []
hedge_port.append(hedge_creation(quintile_port[0],'ep',1,5))
hedge_port.append(hedge_creation(quintile_port[1],'lnsize',1,5))
hedge_port.append(hedge_creation(quintile_port[2],'bk2mkt',5,1))
hedge_port.append(hedge_creation(quintile_port[3],'MOM',5,1))
hedge_port.append(hedge_creation(quintile_port[4],'beta',1,5))
hedge_port.append(hedge_creation(quintile_port[5],'ivol',5,1))

alphas =[]
alpha_t_stats =[]
sharpe_list = []

alphas_ff4 =[]
alphas_ff4_t_stats =[]
with pd.ExcelWriter('dataOutput.xlsx') as writer:
    for i in range (0,6):

        X = hedge_port[i]["mktrf"].values.reshape(-1, 1)
        y = hedge_port[i]["hedge_ret"].values - hedge_port[i].rf
        X_const = sm.add_constant(X)
    # Fit the OLS model
        model = sm.OLS(y, X_const).fit()
    # Extract the intercept and its t-statistic
        intercept = model.params["const"]
        t_stat_intercept = model.tvalues["const"]
        alphas.append(intercept)
        alpha_t_stats.append(t_stat_intercept)

        # for FF 4 model:
        ff3explanatory = sm.add_constant(hedge_port[i][["mktrf", "smb", "hml",'umd']])
        ff3explanatory.reset_index(drop=True, inplace=True)
    # compute sharpe ratio
        sharpe = (hedge_port[i]["hedge_ret"].values - hedge_port[i].rf).mean()/hedge_port[i]["hedge_ret"].std()
        sharpe_list.append(sharpe)

    # run the linear models again
        ff3_model = sm.OLS(y, ff3explanatory).fit()
        intercept = ff3_model.params["const"]
        t_stat_intercept = ff3_model.tvalues["const"]
        alphas_ff4.append(intercept)
        alphas_ff4_t_stats.append(t_stat_intercept)
        #impor to excel
        hedge_port[i].to_excel(writer,f"{factors[i]} hedge portfolio.xlsx")
    merged.to_excel(writer,"part 1-2 data.xlsx")
print(alphas)
print(alpha_t_stats)

print(alphas_ff4)
print(alphas_ff4_t_stats)
print(sharpe_list)




C:\Users\ruixu\AppData\Local\Temp\ipykernel_12148\441793531.py:44: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  hedge_port[i].to_excel(writer,f"{factors[i]} hedge portfolio.xlsx")
C:\Users\ruixu\AppData\Local\Temp\ipykernel_12148\441793531.py:44: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  hedge_port[i].to_excel(writer,f"{factors[i]} hedge portfolio.xlsx")
C:\Users\ruixu\AppData\Local\Temp\ipykernel_12148\441793531.py:44: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  hedge_port[i].to_excel(writer,f"{factors[i]} hedge portfolio.xlsx")
C:\Users\ruixu\AppData\Local\Temp\ipykernel_12148\441793531.py:44: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will

[0.0038756284104023383, 0.02300807381642754, -0.004400540860210706, -0.0013907993889061845, -0.011771832003055854, 0.01633453079082345]
[0.8580197392159133, 6.712654230008553, -1.2232483480121823, -0.30485690852142666, -2.3307031651731434, 3.4299300617061106]
[0.0050441655003027035, 0.023654581179785007, -0.00494934377919773, -0.0012282854587957357, -0.01171736729589615, 0.01631342658918957]
[1.118978361864334, 6.8467754204640086, -1.3732781573245072, -0.26650105830643583, -2.2957628286053473, 3.3897493876775804]
[0.06163517019603266, 0.40162959450179075, -0.07785476954266547, -0.01116197643480072, -0.13375157027728113, 0.19746031242767245]


#### Part 4: unable to shortcell: only long leg: on high ivol

In [107]:
(high_ivol_merged["ret_t1"].values - high_ivol_merged.rf - high_ivol_merged["mktrf"].values).mean()

0.023164314231244994

In [31]:
#hedge port 5 contains ivol port:

high_ivol = quintile_port[5][quintile_port[5].ivol_rank == 5]
high_ivol_merged = pd.merge(high_ivol,ff_data,on=['monthid'])

X = high_ivol_merged["mktrf"].values.reshape(-1, 1)
y = high_ivol_merged["ret_t1"].values - high_ivol_merged.rf
X_const = sm.add_constant(X)
# Fit the OLS model
model = sm.OLS(y, X_const).fit()
# Extract the intercept and its t-statistic
alpha = model.params["const"]
t_stat_alpha = model.tvalues["const"]


# for FF 4 model:
ff3explanatory = sm.add_constant(high_ivol_merged[["mktrf", "smb", "hml",'umd']])
ff3explanatory.reset_index(drop=True, inplace=True)
# dependent is excess return at 2019 dec

# run the linear models again
ff3_model = sm.OLS(y, ff3explanatory).fit()
ff_alpha = ff3_model.params["const"]
t_stat_ff_alpha = ff3_model.tvalues["const"]


print(alpha)
print(t_stat_alpha)
print(ff_alpha)
print(t_stat_ff_alpha)

0.029365927071666683
4.980763870455956
0.029767288144434224
5.002931730549504


In [34]:
# compute annual turnover
print(high_ivol.columns)
turn_over = 0

#for 

Index(['monthid', 'ivol_rank', 'ret_t1', 'ivol'], dtype='object')


In [84]:
merged["ivol_rank"] = merged.groupby("monthid")["ivol_winsorize"].transform(
    lambda x: pd.qcut(x, q=5, labels=False, duplicates="drop") + 1)

# Step 2: Filter to keep only the top rank (5)
group = merged[merged["ivol_rank"] == 5]

# Step 3: Group by month (yyyymm) - ensure the months are in sorted order
group_date = group.groupby("yyyymm")

# Create a dictionary to store turnover values.
# Turnover here is defined as the number of PERMNO that differ from the previous month.
turnover_dict = {}
prev_keys = None

# Sorting the groups by the month key (assuming yyyymm is sortable)
for month, data in sorted(group_date, key=lambda x: x[0]):
    # Get the set of PERMNO keys for the current month.
    curr_keys = set(data['PERMNO'].unique())
    
    # If this is not the first month, compute the symmetric difference.
    if prev_keys is not None:
        # Symmetric difference: keys in prev_keys or curr_keys but not in both.
        diff = curr_keys - prev_keys
        turnover_dict[month] = len(diff)
    else:
        # First month: no previous month to compare, so turnover can be set to None or 0.
        turnover_dict[month] = None
        
    # Update prev_keys for the next iteration.
    prev_keys = curr_keys

# Optionally, you can compute overall summary statistics (e.g., average turnover excluding the first month).
turnover_values = [v for v in turnover_dict.values() if v is not None]
average_turnover = sum(turnover_values) / len(turnover_values) if turnover_values else 0

In [85]:
print(average_turnover)
print(average_turnover/19)
print()

7.953020134228188
0.4185800070646415



In [91]:
average_turnover*12

95.43624161073825

#### Part 5 Fama Macbeth cross sectional asset pricing 

In [ ]:
high_ivol_merged_5 = pd.merge()

In [79]:
group[["beta"]].isna().sum()

beta    297
dtype: int64

In [98]:
clean_group = group.dropna(subset=['beta', "lnsize", "bk2mkt", "ivol"])

ff3explanatory = sm.add_constant(clean_group[["ivol", "beta", "lnsize", "bk2mkt"]]).reset_index(drop=True)
dependent = clean_group["ret_t1"].reset_index(drop=True)

ff3_model = sm.OLS(dependent, ff3explanatory).fit()
print(ff3_model.summary())


                            OLS Regression Results                            
Dep. Variable:                 ret_t1   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     9.483
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           1.25e-07
Time:                        18:31:21   Log-Likelihood:                 1443.4
No. Observations:                4275   AIC:                            -2877.
Df Residuals:                    4270   BIC:                            -2845.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1159      0.018      6.367      0.0